In [ ]:
import logging
import os.path
import pooch

import matplotlib.pyplot as plt
import pyam

from climate_assessment.cli import (
    _get_key_string_and_log_outdir,
    _harmonize_and_infill,
    _load_emissions_convert_to_basic,
    _setup_logging,
)
from climate_assessment.climate import climate_assessment
from climate_assessment.postprocess import do_postprocess

In [ ]:
LOGGER = logging.getLogger("pipeline")
_setup_logging(LOGGER)

## Configuration of input data

### Climate emulator (JSON) configuration data download (FaIR)

Our FaIR setup uses two files, because it reduces the size of the configuration input compared to using only one file.

You can download these files manually from "https://zenodo.org/record/6601980", or you can download it interactively using `pooch`, which we'll showcase here. 

1. First, we specify where we want to put the downloaded cached data (`fair_config_data_dir`)

2. Then, we specify what config files we want, and download them using `pooch.retrieve`

In [ ]:
fair_config_data_dir = os.path.join("..", "data", "fair")

In [ ]:
fair_slim_filename = "fair-1.6.2-wg3-params-slim.json"
fair_common_filename = "fair-1.6.2-wg3-params-common.json"

**N.B.** you only need to run the two cells below this the first time, after that you can also skip and run without internet connection.

In [ ]:
fair_slim_url = "https://zenodo.org/record/6601980/files/fair-1.6.2-wg3-params-slim.json?download=1"
fair_slim_hash = "c071ca619c0ae37a6abdeb79c0cece7b"
    
pooch.retrieve(
            url=fair_slim_url,
            known_hash=f"md5:{fair_slim_hash}",
            path=fair_config_data_dir,
            fname=fair_slim_filename,
        )

In [ ]:
fair_common_url = "https://zenodo.org/record/6601980/files/fair-1.6.2-wg3-params-common.json?download=1"
fair_common_hash = "42ccaffcd3dea88edfca77da0cd5789b"

pooch.retrieve(
            url=fair_common_url,
            known_hash=f"md5:{fair_common_hash}",
            path=fair_config_data_dir,
            fname=fair_common_filename,
        )

### Set general input arguments and options to the climate assessment workflow

In [ ]:
model = "fair"
model_version = "1.6.2"
fair_extra_config = os.path.join(fair_config_data_dir, fair_common_filename)
probabilistic_file = os.path.join(fair_config_data_dir, fair_slim_filename)
num_cfgs = 10  # only 10 configurations (out of 2237 for FaIR) for this example
test_run = True  # this option must be True because we don't run with all 2237 configurations and thus cannot be certain the output temperature data will be correct
scenario_batch_size = 20 # how many scenarios do you want to run in one go?
prefix = "AR6 climate diagnostics" # string prefix 
inputcheck = True # make sure the input data is in the right format

os.environ["ROOT_DIR"] = os.path.join("..")  # set root directory
outdir = "output" # what folder will the climate output be found after the run?


### Choose input emissions pathway file

In [ ]:
EMISSIONS_DATA_DIR = os.path.join(
    "..", "tests", "test-data"
) 
EMISSIONS_INPUT_FILE = "ar6_IPs_emissions.csv"
input_emissions_file = os.path.join(
    EMISSIONS_DATA_DIR, EMISSIONS_INPUT_FILE
)

### Choose infiller database file

In [ ]:
infilling_database = os.path.join(
    "..", "src", "climate_assessment", "infilling", "cmip6-ssps-workflow-emissions.csv"
)

> **_NOTE:_**  The cell above selects the "cmip6-ssps" infiller database, which is directly provided with the climate-assessment package. However, to reproduce the AR6 temperature projections and for most applications, using the AR6 infiller database is strongly recommended (see documentation under "Installation", section "Infiller database"). 

In [ ]:
key_string = _get_key_string_and_log_outdir(
    os.path.join(EMISSIONS_DATA_DIR, EMISSIONS_INPUT_FILE),
    outdir,
    LOGGER,
)

## Run the climate assessment workflow

### Load input emissions file

In [ ]:
input_df = _load_emissions_convert_to_basic(input_emissions_file, LOGGER)

In [ ]:
input_df.timeseries()

### Harmonize and infill the scenarios

*N.B. watch out, the log with information and some warnings will be quite long - but that is nothing to worry about!*

In [ ]:
_harmonize_and_infill(
    input_df,
    inputcheck,
    key_string,
    outdir,
    infilling_database,
    harmonize=True,
    prefix=prefix,
    harmonization_instance="ar6",
)

### Load back in emissions data after infilling was successful

In [ ]:
# read in infilled database
infilled_emissions = os.path.join(
    outdir, "{}_harmonized_infilled.csv".format(key_string)
)
LOGGER.info("Reading in infilled scenarios from: %s", infilled_emissions)
df_infilled = pyam.IamDataFrame(infilled_emissions)

In [ ]:
df_infilled.timeseries()

### Run climate emulator FaIR

*N.B. prepare for another info log*

In [ ]:
df_climate = climate_assessment(
    df_infilled,
    key_string,
    outdir,
    fair_extra_config=fair_extra_config,
    model=model,
    model_version=model_version,
    probabilistic_file=probabilistic_file,
    num_cfgs=num_cfgs,
    historical_warming=0.85,
    historical_warming_reference_period="1850-1900",
    historical_warming_evaluation_period="1995-2014",
    test_run=test_run,
    scenario_batch_size=scenario_batch_size,
    save_raw_output=False,
    co2_and_non_co2_warming=False,
    prefix=prefix,
)

In [ ]:
df_climate.timeseries()

### Combine output and do post-processing

*N.B. prepare yourself for a very long log and a bit of scrolling this time, there will be a lot of (duplicate) information logged for each climate run.*

In [ ]:
results = pyam.concat([df_infilled, df_climate])
output = pyam.concat(
    [input_df.filter(variable=results.variable, keep=False), results]
)
do_postprocess(
    output,
    outdir=outdir,
    key_string=key_string,
    categorisation=True,
    reporting_completeness_categorisation=False,
    prefix=prefix,
    gwp=True,
    model_version=model_version,
    model=model,
)

### Visualise results

In [ ]:
output.filter(variable="*|Surface Temperature (GSAT)|*|50.0th Percentile").plot(color="scenario", fill_between=True, final_ranges=dict(linewidth=5))
plt.tight_layout()
plt.title("Global warming above the 1850-1900 mean")
plt.show()

## Conclusion
That's it! You just ran a a full climate assessment workflow going from emissions to temperature (and more) using the functionality from the climate-assessment package, and then visualised the results. 

Naturally, similar workflows can be constructed using CICERO-SCM (on Linux) and MAGICC (on Windows and Linux)!

It is also possible to run from the command line, and build more elaborate workflows. For that, please see the extended documentation.